<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates the usage of MongoDB Server status Action.
    Using this action, we can query the MongoDB server and find out the server status.
    The sample print function uses matplotlib of python to plot a simple bar chart for
    each of the parameters. 
    <br><br>
    This runbook then goes on to query Long Running queries from the Server.
    Initaites the killop command to kill the Operation IDs that are found to
    be Long Running.</b>
</div>

<br>
<center><h2>MongoDB Server Status</h2></center>

# Steps Overview 
1 <a href="#1"> Connect to Server and get vital statistics </a>
<br>
2 <a href="#2"> Find out all Long running (slow) queries </a>
<br>
3 <a href="#3"> Kill them using the killop command of mongodb </a>

<p id="1"> </p> 

### Connect to Server and get vital statistics
In this Cell, we will use unSkript `mongodb_get_server_status` Lego to get the vital statistics
from the MongoDB Server. Later we use python's matplotlib to plot the statistics like `asserts`,
`connections`, `mem` and `opcounters`. This Lego also prints out the Uptime, Host, Version etc
as well. 


In [32]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
import matplotlib.pyplot as plt

from pydantic import BaseModel
from mongodb.legos.mongodb_util import reachable


from beartype import beartype

def plotData(output, keywords):
    plt.figure(figsize=(24,20), dpi=80)
    row = 0
    col = 0
    for idx, keyword in enumerate(keywords):
        data = output.get(keyword)
        names = list(data.keys())
        values = list(data.values())
        plot = plt.subplot2grid((3,4), (row, col), colspan=1, rowspan=1)
        plot.bar(range(len(data)), values, tick_label=names)
        plot.set_title(keyword)
        col += 1
        if idx % 2:
            row += 1
            col = 0

    plt.show()


def mongodb_get_server_status_printer(output):
    print("Uptime (seconds): ", output.get('uptime'))
    print("UptimeEstimate (seconds): ", output.get('uptimeEstimate'))
    print("Version: ", output.get('version'))
    print("PID : ", output.get('pid'))
    print("Process : ", output.get('process'))
    print("Host : ", output.get('host'))
    print("Local Time: ", output.get('localTime'))

    plotData(output, ['asserts', 'connections', 'mem', 'opcounters'])

@beartype
def mongodb_get_server_status(handle):
    """mongodb_get_server_status returns the mongo server status.

        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.

        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.

        :rtype: Dict with server status.
    """
    # Check the MongoDB 
    try: 
        reachable(handle)
    except Exception as e:
        raise e

    res = handle.admin.command("serverStatus")
    return res


task = Task(Workflow())
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(mongodb_get_server_status, lego_printer=mongodb_get_server_status_printer, hdl=hdl, args=args)

<p id="2"> </p>

### Find out all long running (slow) Queries

Here we will use unSkript `mongodb_list_long_running_queries` Lego. This lego takes  `query_secs_running_threshold`
as input in seconds. This threshold is used to findout all queries that are running More than or equal to `query_secs_running_threshold`
seconds.  

The Queries that are found running for `query_secs_running_threshold` or more seconds will be considered
as long running (slow) queries and this Lego. This lego returns a Dict containing the details of
such long running queries. The output `dict` is stored in `fullOutput` variable to be used later in the cells.

In [31]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from pydantic import BaseModel
from typing import Dict
from tabulate import tabulate
from mongodb.legos.mongodb_util import reachable

from beartype import beartype

def mongodb_list_long_running_queries_printer(output):
    if output is None:
        return
    if len(output.get('inprog')) == 0:
        print("No Long Queries detected") 
        return
    results = [['appName', 'active', 'op', 'ns', 'secs_running', 'desc']]
    try:
        for idx,query in enumerate(output.get('inprog')):
                result = []
                if 'appName' in query.keys():
                    result.append(query.get('appName'))
                else:
                    result.append('None')
                result.append(query.get('active'))
                result.append(query.get('op'))
                result.append(query.get('ns'))
                result.append(query.get('secs_running'))
                result.append(query.get('desc'))
                results.append(result)
    except Exception as e:
        print(f"Exception occured {e}")

    print(tabulate(results, headers="firstrow"))


@beartype
def mongodb_list_long_running_queries(handle, query_secs_running_threshold=5) -> Dict:
    """mongodb_list_long_running_queries  returns information on all the long running queries.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type query_secs_running_threshold: int
        :param query_secs_running_threshold: Integer representing query threshold in seconds.

        :rtype: Result of the query in the Dictionary form.
    """
    # Check the MongoDB 
    try: 
        reachable(handle)
    except Exception as e:
        raise e
    
    try:
        resp = handle.admin.command("currentOp", {"secs_running": {"$gt": query_secs_running_threshold}})
        return resp
    except Exception as e:
        return e


task = Task(Workflow())
task.configure(outputName="fullOutput")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(mongodb_list_queries, lego_printer=mongodb_list_queries_printer, hdl=hdl, args=args)

<p id="3"> </p>

### Kill them using the killop command of mongodb

In this Lego, we will use unSkript `mongodb_kill_queries` Lego to kill all those long running
queries. We use unSkript platform's `iterator` feature to iterate over the list of all Operation IDs
from the previous cell and execute the Kill operation on those IDs. 

This Runbook demonstrates how easy it is to build complex flows such as killing long running
queries using these simple unSkript `Legos`. 

In [39]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from pydantic import BaseModel, Field
from typing import Dict


from beartype import beartype

def mongodb_kill_queries_printer(output):
    if output is None:
        return
    print("\n\n")
    pprint.pprint(output)


def check_id_exists(handle, op_id) -> bool:
    retval = False
    ids = handle.admin.command(({"currentOp": True}))
    current_ids = [o['opid'] for o in ids['inprog']]
    if op_id in current_ids:
        retval = True
    return retval

@beartype
def mongodb_kill_queries(handle, op_id: int) -> Dict:
    """mongodb_kill_queries can kill queries (read operations) that are running on more than one shard in a cluster.

        :type handle: object
        :param handle: Object returned by task.validate(...).

        :type op_id: int
        :param op_id: Operation ID as integer that needs to be killed

        :rtype: Result of the killOp operation for the given op_id in a Dict form.
    """
    # Check the MongoDB 
    try:
        reachable(handle)
    except Exception as e:
        raise e
    try:
        resp = handle.admin.command("killOp", op=op_id)
        if resp.get('ok') == 1:
            # Let us make sure when the KillOp was issued, it 
            # really did kill the query identified by the id.
            # Poll for 10 seconds, if it does not return False
            # raise Exception. else return success message
            try:
                polling2.poll(lambda: check_id_exists(handle, op_id),
                              check_success=polling2.is_value(False),
                              step=1,
                              timeout=10)
                return {'info': f'Successfully Killed OpID {op_id}', 'ok': 1}
            except Exception as e:
                raise e
        else:
            raise Exception("Unable to Get Response from server for killOp command")
    except Exception as e:
        raise e


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "op_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "[k.get('opid') for k in fullOutput.get('inprog')]",
    "iter_parameter": "op_id"
    }''')
task.configure(outputName="killedOutput")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(mongodb_kill_queries, lego_printer=mongodb_kill_queries_printer, hdl=hdl, args=args)

#### Confirming the kill Operation

In [41]:
pprint.pprint(killedOutput)

### Conclusion

In this Runbook, we demonstrated the use of `unSkript's` `iterator` feature coupled with the built-in
MongoDB legos. To view the full platform capabilities of `unSkript` please visit `https://unskript.com` 